## Comparison of various implementations of Householder QR factorization

We implement the Householder algorithm for orthogonal triangularization in three different ways:

- Python and Numpy
- Python and Numpy and Numba (-> full jit compilation, no Python)
- C++ and xtensor and Python/Numpy bindings (pybind11, xtensor-python)

The actual algorithm will be the same in all three cases, only the technology to implement it will be different.

In [1]:
import numpy as np

import numba

In [26]:
# run "pip install --upgrade --force-reinstall --no-deps ." inside the house_cpp directory
# to compile the C++ code and create the Python module, which wraps this code.
import house_cpp

###  Complex-valued test matrices of random shape

We generate random matrices. These matrices have more rows than columns and their entries are complex-valued.

In [2]:
def generate_test_matrices(num=50):
    matrices = []
    
    def generate_matrix(m, n):
        re = np.random.rand(m, n)
        im = np.random.rand(m, n)
        return (re + 1j * im)
    
    for i in range(num):
        two_random_ints = np.random.randint(low=2, high=100, size=2)
        m, n = np.max(two_random_ints), np.min(two_random_ints)
        matrices.append(generate_matrix(m, n))
    
    return matrices

In [3]:
matrices  = generate_test_matrices(num=100)

### Testing the implementations

Each implementation consists of two methods. The first method computes the triangular matrix R by applying Householder reflections to the matrix. The vectors that describe the reflections are additionly returned in a matrix W. The second method computes an orthogonal matrix Q, given W.

In [31]:
def test_implementation(house, formQ, A):
    m, n = A.shape
    assert m >= n
    
    W, R = house(A)
    Q = formQ(W)
    
    assert np.allclose(A, Q[:, :n].dot(R))

In [32]:
def run_comparison(house, formQ):
    for matrix in matrices:
        test_implementation(house, formQ, matrix)

### Python and Numpy implementation

In [24]:
def house_numpy(A):
    """
    Computes an implicit representation of a full QR factorization A = QR
    of an m x n matrix A with m ≥ n using Householder reﬂections.
    
    Returns
    -------
    - lower-triangular matrix W ∈ C m×n whose columns are the vectors v_k
        deﬁning the successive Householder reﬂections
    - triangular matrix R ∈ C n x n
    """
    m, n = A.shape
    assert m >= n
    
    R = np.copy(A).astype(complex)
    W = np.zeros_like(R, dtype=complex)
    
    for k in range(n):
        v_k = np.copy(R[k:, k])
        sgn = np.sign(v_k[0])
        if  sgn == 0: sgn = 1
        v_k[0] += np.exp(1j*np.angle(v_k[0])) * sgn * np.linalg.norm(v_k)
        v_k /= np.linalg.norm(v_k)
        W[k:, k] = v_k
        R[k:, k:] -= 2 * np.outer(v_k, np.dot(v_k.conj().T, R[k:, k:]))           # 124 ms
        #R[k:, k:] -= 2 * np.dot(np.outer(v_k, v_k.conj().T), R[k:, k:]) # slower # 155 ms
    if m > n:
        R = np.copy(R[:n,:])
    return W, R

In [25]:
def formQ_numpy(W):
    """ generates a corresponding m × m orthogonal matrix Q
    """
    m, n = W.shape
    Q = np.eye(m, dtype=complex)
    
    for i in range(n):
        for k in range(n-1, -1, -1):
            v_k = W[k:, k]
            Q[k:, i] -= 2 * v_k * np.dot(v_k.conjugate(), Q[k:, i])
    return Q

In [33]:
%timeit run_comparison(house_numpy, formQ_numpy)

1.4 s ± 9.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Python and Numpy and Numba implementation

In [28]:
@numba.jit(numba.types.UniTuple(numba.complex128[:,:], 2)(numba.complex128[:,:]), nopython=True)
def house_numba(A):
    """
    Computes an implicit representation of a full QR factorization A = QR
    of an m x n matrix A with m ≥ n using Householder reﬂections.
    
    Returns
    -------
    - lower-triangular matrix W ∈ C m×n whose columns are the vectors v_k
        deﬁning the successive Householder reﬂections
    - upper-triangular matrix R ∈ C n x n
    """
    m, n = A.shape
    assert m >= n
    
    R = np.copy(A)
    W = np.zeros_like(R, dtype=numba.complex128)
    
    for k in range(n):
        v_k = np.copy(R[k:, k])
        sgn = np.sign(v_k[0])
        if  sgn == 0: sgn = 1
        v_k[0] += np.exp(1j*np.angle(v_k[0])) * sgn * np.linalg.norm(v_k)
        v_k /= np.linalg.norm(v_k)
        W[k:, k] = v_k
        R[k:, k:] -= 2 * np.outer(v_k, np.dot(np.conjugate(v_k).T, R[k:, k:]))          # 28   ms
        #R[k:, k:] -= 2 * np.dot(np.outer(v_k, np.conjugate(v_k).T), R[k:, k:])  # slower 31.5 ms
    if m > n:
        R = np.copy(R[:n,:])
    return W, R

In [29]:
@numba.jit(numba.complex128[:,:](numba.complex128[:,:]), nopython=True)
def formQ_numba(W):
    """
    Generates a corresponding m × m orthogonal matrix Q.
    """
    m, n = W.shape
    #np.eye(m, dtype=complex128) does not work
    Q = np.zeros((m, m), dtype=numba.complex128)
    for i in range(m):
        Q[i, i] = 1
    
    for i in range(n):
        for k in range(n-1, -1, -1):
            v_k = W[k:, k]
            Q[k:, i] -= 2 * v_k * np.dot(np.conjugate(v_k), Q[k:, i])
    return Q

In [34]:
%timeit run_comparison(house_numba, formQ_numba)

570 ms ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### C++ and xtensor implementation

The C++ source code is in ./house_cpp/src/main.cpp

In [22]:
# as of now only the formQ method works

def compute_Ws():
    Ws = []
    for i in range(len(matrices)):
        W, R = house(matrices[i])
        Ws.append(W)
    return Ws

Ws = compute_Ws()

In [40]:
def compute_Qs(formQ):
    for W in Ws:
        _ = formQ(W)

In [41]:
%timeit compute_Qs(formQ_numba)

477 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
# yeah my first dive into C++ got awarded. 23% of runtime saved
%timeit compute_Qs(house_cpp.formQ)

363 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
W1, R1 = house_cpp.house(matrices[1])

(88, 78)